In [1]:
import xarray as xr

In [2]:
time_lim = ['1869','2016']

In [5]:
data_name = ['sst','ice']
filename = {}
filename['sst'] = '/projects/GEOCLIM/hengquan/models_stellar/AM4/hadisst/hadisst_sst.data.nc'
filename['ice'] = '/projects/GEOCLIM/hengquan/models_stellar/AM4/hadisst/hadisst_ice.data.nc'
for var in data_name:
    ds = xr.open_dataset(filename[var])
    # print(ds)

    time = ds.time
    new_time = time.sel(time=slice('1890-01-16', '2009-12-16'))
    sst = ds[var]
    sst_1982_2001 = sst.sel(time=slice(time_lim[0], time_lim[1]))
    # 1982-2001
    # sst_1982_2001_clim = sst_1982_2001.groupby('time.month').mean('time')
    # whole period
    sst_1982_2001_clim = sst.groupby('time.month').mean('time')
    da = sst_1982_2001_clim
    da = xr.concat([da,]*120, dim='month')
    da = da.rename({'month':'time'})
    da['time'] = new_time
    da.attrs = sst.attrs

    yr = xr.DataArray(da.time.dt.year.values, dims=['time'], coords=[new_time])
    yr.attrs = dict(long_name='year')
    mo = xr.DataArray(da.time.dt.month.values, dims=['time'], coords=[new_time])
    mo.attrs = dict(long_name='month')
    dy = xr.DataArray(da.time.dt.day.values, dims=['time'], coords=[new_time])
    dy.attrs = dict(long_name='day')
    nrecords = da.time.size
    lon_bnds = ds.lon_bnds
    lat_bnds = ds.lat_bnds
    ds_new = xr.Dataset({var: da,
        'year': yr,
        'mon': mo,
        'day': dy,
        'lon_bnds': lon_bnds,
        'lat_bnds': lat_bnds,                 
        'nrecords': nrecords})
    ds_new.attrs = ds.attrs 

    bnddim,xdim,ydim,tdim ='bnds','lon','lat','time'
    # nc_encoding = {bnddim:{'dtype': 'double', '_FillValue': False},
    #             tdim:{'dtype': 'double', '_FillValue': False},
    #             xdim:{'_FillValue': False},
    #             ydim:{'_FillValue': False},
    #             'lon_bnds':{'dtype':'float64', '_FillValue': False}, 
    #             'lat_bnds':{'dtype':'float64', '_FillValue': False},
    #             'sst':{'_FillValue': False},
    #             'mon': {'dtype': 'float32', '_FillValue': False},
    #             'day': {'dtype': 'float32', '_FillValue': False},
    #             'year': {'dtype': 'float32', '_FillValue': False},
    #             'nrecords': {'dtype': 'int32', '_FillValue': False},
    #             }
    #ds_new.to_netcdf('/projects/GEOCLIM/hengquan/models_stellar/AM4/hadisst/hadisst_sst_ctl.data.nc', unlimited_dims=tdim, encoding=nc_encoding)
    ds_new.to_netcdf(f'hadisst_{var}.clm.{time_lim[0]}_{time_lim[1]}.nc', unlimited_dims=tdim)


In [6]:
ds_check = xr.open_dataset(f'hadisst_sst.clm.{time_lim[0]}_{time_lim[1]}.nc')
print(ds_check)

<xarray.Dataset>
Dimensions:   (lon: 360, lat: 180, time: 1440, bnds: 2)
Coordinates:
  * lon       (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat       (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * time      (time) datetime64[ns] 1890-01-16T12:00:00 ... 2009-12-16T12:00:00
Dimensions without coordinates: bnds
Data variables:
    sst       (time, lat, lon) float32 ...
    year      (time) int64 ...
    mon       (time) int64 ...
    day       (time) int64 ...
    lon_bnds  (lon, bnds) float64 ...
    lat_bnds  (lat, bnds) float64 ...
    nrecords  int64 ...
Attributes: (12/35)
    CDI:                     Climate Data Interface version 1.7.2 (http://mpi...
    source:                  PCMDI-AMIP 1.1.0: Merged SST based on UK MetOffi...
    institution:             Program for Climate Model Diagnosis and Intercom...
    Conventions:             CF-1.7 CMIP-6.0
    activity_id:             input4MIPs
    comment:                 Based on Hurrell

In [7]:
## import self defined functions
from sys import path 
# insert at 1, 0 is the script path (or '' in REPL)
path.insert(1, '/tigress/cw55/local/python_lib')
from cg_funcs import global_mean_xarray
# root = '/tigress/cw55/work/2022_radi_nn/NN_AM4'
# path.insert(1,  root)
# path.insert(1,  root+'/work')
# # import work.lw_csaf_Li5Relu_EN.train_script01 as lwcsafen
# import lw_csaf_Li5Relu_EY.train_script01 as lwcsafey 
# # import work.lw_af_Li5Relu_EN.train_script01 as lwafen
# import lw_af_Li5Relu_EY.train_script01 as lwafey 

In [8]:
global_mean_xarray(ds_check.sst.mean('time'))

<xarray.DataArray ()>
array(290.45398964)

In [9]:
global_mean_xarray(ds_new.sst.mean('time'))

AttributeError: 'Dataset' object has no attribute 'sst'

In [ ]:
global_mean_xarray(ds.sst.sel(time=slice('1982-01-16', '2001-12-16')).mean('time'))